#Extended Kalman Filter Example

###Introduction

This notebook is designed to demonstrate how to use the StateSpace.jl package to execute the Extended Kalman filter for a non-linear State Space model. The example that has been used here closely follows Markus Gesmann's
Mages' blog titled [Extended Kalman filter example in R](http://www.magesblog.com/2015/01/extended-kalman-filter-example-in-r.html) which in turn
was based on a blog post titled [Fun with (Extended Kalman) Filters](https://idontgetoutmuch.wordpress.com/2014/09/09/fun-with-extended-kalman-filters-4/)
by Dominic Steinitz.

For those of you that do not need/want the explanation of the model and the code, you can skip right to the end of this notebook where the entire section of code required to run this example is given.

###The Problem
The problem that we will consider here is that of predicting the true value of a population (say of bacteria, but it could be anything) from some noisy measurements of the population. We also would like to predict the growth rate of the population, however we are faced with the problem that we do not observe the growth rate

#####Process  Model
It is assumed that the population evolves according to the logistic growth model. This is written mathematically as an ordinary differential equation:

$$\frac{\mathrm{d}p}{\mathrm{d}t} = rp\left(1 - \frac{p}{k} \right),$$

where $p$ is the population number, $t$ is the time, and $k$ is the carrying capacity.   
The solution to the above differetial equation is given by

$$p(t) = \frac{kp_0\exp(rt)}{k+p_0[\exp(rt) - 1]},$$

where $p_0$ is the initial population.   
This is almost in the form that we need. The only problem is that it needs to be formulated in a discrete manner whereby the population at the current time, $p_n$, is written in terms of the population and growth rate at the previous state, $p_{n-1}$ and $r_{n-1}$ respectively. This can be done like so:

$$p_n = \frac{kp_{n-1}\exp(r_{n-1}\Delta t)}{k+p_{n-1}[\exp(r_{n-1}\Delta t) - 1]},$$

where $\Delta t$ is the change in time from one measurement of the population to the next. This is our transistion (process) equation for the population.   

As mentioned previously, we also want to know the growth rate, and hence it is both the population number and the growth rate value together that define the state of our system. Hence we need to also define how the growth rate evolves. Here we will simply assume that the growth rate stays constant. Hence the transition equation for the growth rate will be written as:

$$r_n = r_{n-1}.$$

We will assume that these processes experience some noise. This will be in the form of zero mean Gaussian noise. We'll set the variance at 0.001 for both of these processes and hence we can write the full process equations as:

$$p_n = \frac{kp_{n-1}\exp(r_{n-1}\Delta t)}{k+p_{n-1}[\exp(r_{n-1}\Delta t) - 1]} + V_n \qquad V_n \sim \mathcal{N}(0, 0.001)$$

$$r_n = r_{t-n} + W_n \qquad W_n \sim \mathcal{N}(0, 0.001)$$

#####Observation Model
We assume that we observe the population directly but there is some zero mean Gaussian noise with variance = 25. Hence our observation model is:

$$y_n = p_n + Q_n \qquad Q_n \sim \mathcal{N}(0, 25),$$

where $y_n$ is the observation at the current time step.   

We do not observe the growth rate :(.

###Setting up the problem
First we'll import the required modules

In [1]:
using StateSpace
using Distributions
using Gadfly
using DataFrames
using Colors

#####Generate noisy observations
First thing we'll do is create the logistic function required to describe the evolution of the population

In [2]:
function logisticGrowth(r, p, k, t)
    k * p * exp(r*t) / (k + p * (exp(r*t) - 1))
end
logisticGrowth(state) = logisticGrowth(state[1], state[2], k, Δt)

logisticGrowth (generic function with 2 methods)

The second definition of the logistic function will come in useful when we are defining the process function.

Let's set the values of the parameters that we're going to use for the logistic function

In [3]:
r = 0.2
k = 100.0
p0 = 0.1 * k
Δt = 0.1

0.1

Now we'll set the variance of the observations (measurement noise)

In [4]:
measurement_noise_variance = 25.0

25.0

We can now define the number of observations and generate the true and observed population values. Note that we don't actually observe the growth rate value so we'll set the corresponding values in the vector to 'NaN'. *For more information on handling missing observations in this package check out the [missing observations example](https://github.com/JonnyCBB/StateSpace.jl/blob/MissingObs/examples/MissingObservationsExample.ipynb).*

In [5]:
numObs = 100
true_values = Vector{Float64}(numObs)
population_measurements = Vector{Float64}(numObs)
growth_rate_measurements = Vector{Float64}(numObs)
for i in 1:numObs
    true_values[i] = logisticGrowth(r, p0, k, i*Δt)
    population_measurements[i] = true_values[i] + randn() * sqrt(measurement_noise_variance)
    growth_rate_measurements[i] = NaN
end

Finally we need to concatenate both population and growth rate arrays and ensure that have the correct dimensions

In [6]:
measurements = [growth_rate_measurements population_measurements]'

2x100 Array{Float64,2}:
 NaN        NaN       NaN        NaN        …  NaN       NaN       NaN     
   4.95685    5.6229    9.28264    6.86086      45.9464   44.1878   44.3974

#####Define Kalman Filter Parameters
We now need to define the process and observation model according to the model described earlier.

In [7]:
function process_fcn(state)
    predict_growth_rate = state[1]
    predict_population = logisticGrowth(state[1], state[2], k, Δt)
    new_state = [predict_growth_rate, predict_population]
    return new_state
end
process_noise_mat = diagm([0.001, 0.001])

function observation_fcn(state)
    growth_rate_observation = 0.0
    population_observation = 1.0 * state[2]
    observation = [growth_rate_observation, population_observation]
    return observation
end
observation_noise_mat = diagm([1.0, measurement_noise_variance])

2x2 Array{Float64,2}:
 1.0   0.0
 0.0  25.0

some couple of things to note about the above code:   
-The process of the growth and population are assumed uncorrelated, hence the off **diagonal terms are zero**. This is why the process noise matrix (and the observation noise matrix) are created as diagonal matrices.   
-Although the growth rate is not observed, we've still given the observation noise for the growth rate a **non-zero** value. It could be any non-zero value, I've just picked 1.0 (It shouldn't change the overall results). The problem with setting this value to zero is that you end up getting a singularity and hence the matrix algebra doesn't work and it will crash. So it must be a non zero value.   

We can now create an instance of the Nonlinear State Space Model

In [8]:
nonLinSSM = NonlinearGaussianSSM(process_fcn, process_noise_mat, observation_fcn, observation_noise_mat)

StateSpace.NonlinearGaussianSSM{Float64}(process_fcn,j,2x2 Array{Float64,2}:
 0.001  0.0  
 0.0    0.001,observation_fcn,j,2x2 Array{Float64,2}:
 1.0   0.0
 0.0  25.0)

#####Initial Guess
Now we can create an initial guess for the growth rate and the initial population along with the corresponding covariance matrix. 

In [9]:
initial_guess = MvNormal([0.5, 10], diagm([1.0,20.0]))

FullNormal(
dim: 2
μ: [0.5,10.0]
Σ: 2x2 Array{Float64,2}:
 1.0   0.0
 0.0  20.0
)


#####Perform Extended Kalman Filter Algorithm
Now we have all of the parameters:
1. noisy observations
2. process (transition) and observation (emission) model paramaters
3. initial guess of state   

We can use the Kalman Filter to predict the true underlying state (growth rate and population).

In [10]:
filtered_state = filter(nonLinSSM, measurements, initial_guess, true)

SmoothedState{Float64}

The last argument in the filter - 'true' - allows us to use the information we have about the observations that we actually observed. It also estimates the observations that we didn't observe using our process and observation model. *For more information on handling missing observations in this package check out the [missing observations example](https://github.com/JonnyCBB/StateSpace.jl/blob/MissingObs/examples/MissingObservationsExample.ipynb).*

###Plot the results
Now we will plot the results. First we will look at the population estimates

First we extract the filtered population values along with their corresponding $2\sigma$ values (the give the area that we would expect the true value to lie with 95% confidence)

In [11]:
x_data = 1:numObs
population_array = Vector{Float64}(numObs+1)
confidence_array = Vector{Float64}(numObs+1)
population_array[1] = initial_guess.μ[2]
confidence_array[1] = 2*sqrt(initial_guess.Σ.mat[2,2])
for i in x_data
    current_state = filtered_state.state[i]
    population_array[i+1] = current_state.μ[2]
    confidence_array[i+1] = 2*sqrt(current_state.Σ.mat[2,2])
end


101 estimates of 2-D process from 2-D observations
Log-likelihood: -436.93441404047263


Next we will create a dataframe. This is simply so the syntax is simple for plotting the ribbon digram which will represent the state along with the confidence interval

In [12]:
df_fs = DataFrame(
    x = [0;x_data],
    y = population_array,
    ymin = population_array - confidence_array,
    ymax = population_array + confidence_array,
    f = "Filtered values"
    )

,x,y,ymin,ymax,f
1,0,10.0,1.0557280900008408,18.94427190999916,Filtered values
2,1,10.0,1.0557280900008408,18.94427190999916,Filtered values
3,2,8.068912500755761,1.1837120780073276,14.954112923504194,Filtered values
4,3,7.5479131162220146,1.5575048931095132,13.538321339334516,Filtered values
5,4,8.350976318769114,2.808563345875587,13.89338929166264,Filtered values
6,5,8.275071661696597,2.861745259439573,13.68839806395362,Filtered values
7,6,11.511777536000075,6.151845219263289,16.87170985273686,Filtered values
8,7,14.311784334288365,8.618260871160278,20.005307797416453,Filtered values
9,8,14.470772189923796,8.415108874369587,20.526435505478005,Filtered values
10,9,13.03329620745293,6.939638655951538,19.126953758954322,Filtered values


Next we will create separate colours for the observations and the true value. Here we will generate the default distinguishable colors used by Gadfly.

In [13]:
n = 3 #We will require 3 different colors
getColors = distinguishable_colors(n, Color[LCHab(70, 60, 240)],
                                   transform=c -> deuteranopic(c, 0.5),
                                   lchoices=Float64[65, 70, 75, 80],
                                   cchoices=Float64[0, 50, 60, 70],
                                   hchoices=linspace(0, 330, 24))

Finally we will plot the results of the filtered population estimates

In [14]:
population_state_plot = plot(
    layer(x=0:numObs, y=[p0;measurements[2,:]'], Geom.point, Theme(default_color=getColors[2])),
    layer(x=0:numObs, y=[p0;true_values], Geom.line, Theme(default_color=getColors[3])),
    layer(df_fs, x=:x, y=:y, ymin=:ymin, ymax=:ymax, Geom.line, Geom.ribbon),
    Guide.xlabel("Measurement Number"), Guide.ylabel("Population"),
    Guide.manual_color_key("Colour Key",["Filtered Estimate", "Measurements","True Value "],[getColors[1],getColors[2],getColors[3]]),
    Guide.title("Extended Kalman Filter Example")
    )
display(population_state_plot)

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 Measurement Number 
 
 
 -150 
 -100 
 -50 
 0 
 50 
 100 
 150 
 200 
 250 
 -100 
 -95 
 -90 
 -85 
 -80 
 -75 
 -70 
 -65 
 -60 
 -55 
 -50 
 -45 
 -40 
 -35 
 -30 
 -25 
 -20 
 -15 
 -10 
 -5 
 0 
 5 
 10 
 15 
 20 
 25 
 30 
 35 
 40 
 45 
 50 
 55 
 60 
 65 
 70 
 75 
 80 
 85 
 90 
 95 
 100 
 105 
 110 
 115 
 120 
 125 
 130 
 135 
 140 
 145 
 150 
 155 
 160 
 165 
 170 
 175 
 180 
 185 
 190 
 195 
 200 
 -100 
 0 
 100 
 200 
 -100 
 -90 
 -80 
 -70 
 -60 
 -50 
 -40 
 -30 
 -20 
 -10 
 0 
 10 
 20 
 30 
 40 
 50 
 60 
 70 
 80 
 90 
 100 
 110 
 120 
 130 
 140 
 150 
 160 
 170 
 180 
 190 
 200 
 
 
 
 Filtered Estimate 
 Measurements 
 True Value 
 
 
 
 
 
 
 
 Colour Key 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M110.01,28.1 L 109.11 28.71 108.21 29.8 107.31 30.3 106.41 29.44 105.51 29.53 104.61 29.98 103.7 31.62 102.8 32.15 101.9 32.29 101 31.28 100.1 33.01 99.2 32.68 98.3 33.56 97.4 34.6 96.5 35.5 95.6 36.41 94.7 37.5 93.8 37.24 92.9 36.94 92 39.12 91.1 39.64 90.2 39.68 89.3 39.12 88.4 38.41 87.5 40.09 86.6 39.69 85.69 40.63 84.79 41.57 83.89 41.03 82.99 41.56 82.09 42.83 81.19 42.01 80.29 41.55 79.39 42.72 78.49 42.99 77.59 44.44 76.69 44.83 75.79 44.29 74.89 45.4 73.99 46.34 73.09 47.39 72.19 49.06 71.29 49.69 70.39 49.8 69.49 50.71 68.59 50.68 67.68 51.08 66.78 52.92 65.88 52.83 64.98 53.34 64.08 54.92 63.18 55.76 62.28 56.6 61.38 57.16 60.48 56.83 59.58 58.04 58.68 59.84 57.78 60.2 56.88 62.27 55.98 63.38 55.08 63.46 54.18 63.54 53.28 65 52.38 65.84 51.48 67.14 50.58 66.36 49.67 66.46 48.77 67.51 47.87 69.07 46.97 70.02 46.07 70.56 45.17 71.42 44.27 73.19 43.37 72.19 42.47 72.25 41.57 71.47 40.67 71.12 39.77 69.3 38.87 69.91 37.97 68.78 37.07 66.27 36.17 66.67 35.27 64.4 34.37 63.56 33.47 62.06 32.57 65.42 31.66 65.94 30.76 67.09 29.86 68.95 28.96 70.83 28.06 69.82 27.16 67.93 26.26 67.67 25.36 70.83 24.46 75.05 23.56 75.11 22.66 76.72 21.76 77.2 20.86 77.36 19.96 77.36 19.96 54.43 20.86 54.43 21.76 59.54 22.66 61.36 23.56 60.9 24.46 61.17 25.36 57.08 26.26 53.07 27.16 52.4 28.06 54.19 28.96 55.78 29.86 54.64 30.76 53.15 31.66 52.19 32.57 51.85 33.47 48.73 34.37 50.26 35.27 51.4 36.17 54.06 37.07 54.19 37.97 57.12 38.87 58.77 39.77 58.67 40.67 60.86 41.57 61.64 42.47 62.79 43.37 63.09 44.27 64.38 45.17 62.92 46.07 62.19 46.97 61.71 47.87 60.77 48.77 59.16 49.67 57.97 50.58 57.7 51.48 58.33 52.38 56.97 53.28 56.01 54.18 54.43 55.08 54.19 55.98 54 56.88 52.81 57.78 50.65 58.68 50.15 59.58 48.25 60.48 46.93 61.38 47.15 62.28 46.55 63.18 45.68 64.08 44.82 64.98 43.22 65.88 42.67 66.78 42.75 67.68 40.91 68.59 40.5 69.49 40.52 70.39 39.62 71.29 39.51 72.19 38.9 73.09 37.23 73.99 36.17 74.89 35.21 75.79 34.07 76.69 34.58 77.59 34.2 78.49 32.75 79.39 32.46 80.29 31.28 81.19 31.73 82.09 32.56 82.99 31.31 83.89 30.78 84.79 31.32 85.69 30.39 86.6 29.45 87.5 29.84 88.4 28.17 89.3 28.87 90.2 29.43 91.1 29.4 92 28.89 92.9 26.72 93.8 26.99 94.7 27.25 95.6 26.16 96.5 25.23 97.4 24.31 98.3 23.26 99.2 22.36 100.1 22.67 101 20.93 101.9 21.93 102.8 21.79 103.7 21.26 104.61 19.62 105.51 19.16 106.41 19.07 107.31 19.93 108.21 19.43 109.11 18.34 110.01 17.74 z"/>
 
 
 <path fill="none" d="M19.96,65.89 L 20.86 65.89 21.76 68.37 22.66 69.04 23.56 68.01 24.46 68.11 25.36 63.96 26.26 60.37 27.16 60.16 28.06 62.01 28.96 63.31 29.86 61.79 30.76 60.12 31.66 59.06 32.57 58.63 33.47 55.39 34.37 56.91 35.27 57.9 36.17 60.36 37.07 60.23 37.97 62.95 38.87 64.34 39.77 63.99 40.67 65.99 41.57 66.56 42.47 67.52 43.37 67.64 44.27 68.79 45.17 

We can use a similar approach to plot the filtered growth rate values

In [15]:
x_data = 1:numObs
growth_rate_array = Vector{Float64}(numObs+1)
confidence_array = Vector{Float64}(numObs+1)
growth_rate_array[1] = initial_guess.μ[1]
confidence_array[1] = initial_guess.Σ.mat[1,1]
for i in x_data
    current_state = filtered_state.state[i]
    growth_rate_array[i+1] = current_state.μ[1]
    confidence_array[i+1] = 2*sqrt(current_state.Σ.mat[1,1])
end
df_fs = DataFrame(
    x = [0;x_data],
    y = growth_rate_array,
    ymin = growth_rate_array - confidence_array,
    ymax = growth_rate_array + confidence_array,
    f = "Filtered values"
    )

growth_rate_state_plot = plot(
    layer(x=0:numObs, y=ones(numObs+1)*r, Geom.line, Theme(default_color=getColors[3])),
    layer(df_fs, x=:x, y=:y, ymin=:ymin, ymax=:ymax, Geom.line, Geom.ribbon),
    Guide.xlabel("Measurement Number"), Guide.ylabel("Growth Rate"),
    Guide.manual_color_key("Colour Key",["Filtered Estimate", "Measurements","True Value "],[getColors[1],getColors[2],getColors[3]]),
    Guide.title("Extended Kalman Filter Example")
    )
display(growth_rate_state_plot)

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 Measurement Number 
 
 
 -150 
 -100 
 -50 
 0 
 50 
 100 
 150 
 200 
 250 
 -100 
 -95 
 -90 
 -85 
 -80 
 -75 
 -70 
 -65 
 -60 
 -55 
 -50 
 -45 
 -40 
 -35 
 -30 
 -25 
 -20 
 -15 
 -10 
 -5 
 0 
 5 
 10 
 15 
 20 
 25 
 30 
 35 
 40 
 45 
 50 
 55 
 60 
 65 
 70 
 75 
 80 
 85 
 90 
 95 
 100 
 105 
 110 
 115 
 120 
 125 
 130 
 135 
 140 
 145 
 150 
 155 
 160 
 165 
 170 
 175 
 180 
 185 
 190 
 195 
 200 
 -100 
 0 
 100 
 200 
 -100 
 -90 
 -80 
 -70 
 -60 
 -50 
 -40 
 -30 
 -20 
 -10 
 0 
 10 
 20 
 30 
 40 
 50 
 60 
 70 
 80 
 90 
 100 
 110 
 120 
 130 
 140 
 150 
 160 
 170 
 180 
 190 
 200 
 
 
 
 Filtered Estimate 
 Measurements 
 True Value 
 
 
 
 
 
 
 
 Colour Key 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M110.01,54.32 L 109.1 54.39 108.2 54.65 107.29 54.7 106.38 54.22 105.48 54.06 104.57 54.03 103.67 54.47 102.76 54.51 101.85 54.4 100.95 53.81 100.04 54.25 99.13 53.91 98.23 54.03 97.32 54.22 96.42 54.38 95.51 54.56 94.6 54.84 93.7 54.59 92.79 54.29 91.89 54.99 90.98 55.08 90.07 54.99 89.17 54.63 88.26 54.17 87.36 54.63 86.45 54.29 85.54 54.47 84.64 54.67 83.73 54.27 82.83 54.27 81.92 54.57 81.01 54.04 80.11 53.59 79.2 53.76 78.29 53.58 77.39 53.87 76.48 53.75 75.58 53.21 74.67 53.32 73.76 53.36 72.86 53.45 71.95 53.83 71.05 53.81 70.14 53.56 69.23 53.62 68.33 53.28 67.42 53.08 66.52 53.52 65.61 53.13 64.7 52.97 63.8 53.3 62.89 53.33 61.98 53.38 61.08 53.29 60.17 52.74 59.27 52.92 58.36 53.47 57.45 53.31 56.55 54.14 55.64 54.53 54.74 54.36 53.83 54.17 52.92 54.85 52.02 55.24 51.11 55.98 50.21 55.41 49.3 55.37 48.39 56.02 47.49 57.12 46.58 57.88 45.68 58.41 44.77 59.19 43.86 60.62 42.96 60.23 42.05 60.54 41.14 60.33 40.24 60.41 39.33 59.54 38.43 60.25 37.52 59.84 36.61 58.4 35.71 58.91 34.8 57.35 33.9 56.61 32.99 54.92 32.08 57.83 31.18 58.28 30.27 59.58 29.37 62.14 28.46 65.31 27.55 64.62 26.65 61.53 25.74 59.44 24.83 64.4 23.93 72.81 23.02 72.76 22.12 74.3 21.21 73.57 20.3 72.3 19.4 59.48 19.4 33.84 20.3 21.02 21.21 22.74 22.12 24.51 23.02 24.62 23.93 27.12 24.83 21.69 25.74 20.85 26.65 27.89 27.55 35.59 28.46 39.94 29.37 39.62 30.27 39.45 31.18 40.25 32.08 41.62 32.99 40.27 33.9 43.35 34.8 45.26 35.71 47.77 36.61 48.01 37.52 50.07 38.43 50.96 39.33 50.62 40.24 51.79 41.14 51.95 42.05 52.34 42.96 52.16 43.86 52.66 44.77 51.32 45.68 50.6 46.58 50.14 47.49 49.44 48.39 48.39 49.3 47.82 50.21 47.94 51.11 48.6 52.02 47.93 52.92 47.64 53.83 47.05 54.74 47.33 55.64 47.6 56.55 47.29 57.45 46.56 58.36 46.81 59.27 46.34 60.17 46.25 61.08 46.88 61.98 47.04 62.89 47.07 63.8 47.1 64.7 46.82 65.61 47.03 66.52 47.47 67.42 47.07 68.33 47.3 69.23 47.68 70.14 47.64 71.05 47.92 71.95 47.96 72.86 47.6 73.76 47.52 74.67 47.5 75.58 47.42 76.48 47.97 77.39 48.11 78.29 47.83 79.2 48.03 80.11 47.87 81.01 48.34 81.92 48.88 82.83 48.59 83.73 48.6 84.64 49.01 85.54 48.82 86.45 48.65 87.36 49 88.26 48.54 89.17 49.01 90.07 49.37 90.98 49.47 91.89 49.38 92.79 48.69 93.7 48.99 94.6 49.25 95.51 48.97 96.42 48.8 97.32 48.65 98.23 48.46 99.13 48.36 100.04 48.7 100.95 48.27 101.85 48.86 102.76 48.98 103.67 48.95 104.57 48.52 105.48 48.56 106.38 48.72 107.29 49.2 108.2 49.15 109.1 48.9 110.01 48.84 z"/>
 
 
 <path fill="none" d="M19.4,46.66 L 20.3 46.66 21.21 48.16 22.12 49.4 23.02 48.69 23.93 49.97 24.83 43.05 25.74 40.14 26.65 44.71 27.55 50.11 28.46 52.62 29.37 50.88 30.27 49.52 31.18 49.27 32.08 49.72 32.99 47.59 33.9 49.98 34.8 51.3 35.71 53.34 36.61 53.2 37.52 54.95 38.43 55.6 39.33 55.08 40.24 56.1 41.14 56.14 42.05 56.44 42.96 5

For those that just want the code, without the explanation, you have come to the right place. Here it is:

In [17]:
using StateSpace
using Distributions
using Gadfly
using DataFrames
using Colors

r = 0.2 #r is the growth rate
k = 100.0 #k is the carrying capacity
p0 = 0.1 * k # p0 is the initial population
Δt = 0.1 # Δt is the change in time.

#Define the logistic growth function to set the observations
function logisticGrowth(r, p, k, t)
    k * p * exp(r*t) / (k + p * (exp(r*t) - 1))
end
logisticGrowth(state) = logisticGrowth(state[1], state[2], k, Δt)

#Generate noisy measurements
measurement_noise_variance = 25.0
numObs = 100
true_values = Vector{Float64}(numObs)
population_measurements = Vector{Float64}(numObs)
growth_rate_measurements = Vector{Float64}(numObs)
for i in 1:numObs
    true_values[i] = logisticGrowth(r, p0, k, i*Δt)
    population_measurements[i] = true_values[i] + randn() * sqrt(measurement_noise_variance)
    growth_rate_measurements[i] = NaN
end
measurements = [growth_rate_measurements population_measurements]'

#Section: Describe Extended Kalman Filter parameters
function process_fcn(state)
    predict_growth_rate = state[1]
    predict_population = logisticGrowth(state)
    new_state = [predict_growth_rate, predict_population]
    return new_state
end
process_noise_mat = diagm([0.001, 0.001])

function observation_fcn(state)
    growth_rate_observation = 0.0
    population_observation = 1.0 * state[2]
    observation = [growth_rate_observation, population_observation]
    return observation
end
observation_noise_mat = diagm([1.0, measurement_noise_variance])

#Create instance of our EKF model
nonLinSSM = NonlinearGaussianSSM(process_fcn, process_noise_mat, observation_fcn, observation_noise_mat)

initial_guess = MvNormal([0.5, 10], diagm([1.0,20.0])) #Set initial guess of the state

filtered_state = filter(nonLinSSM, measurements, initial_guess, true) #Execute  the Extended Kalman Filter

#Plot Filtered results for population
x_data = 1:numObs
population_array = Vector{Float64}(numObs+1)
confidence_array = Vector{Float64}(numObs+1)
population_array[1] = initial_guess.μ[2]
confidence_array[1] = 2*sqrt(initial_guess.Σ.mat[2,2])
for i in x_data
    current_state = filtered_state.state[i]
    population_array[i+1] = current_state.μ[2]
    confidence_array[i+1] = 2*sqrt(current_state.Σ.mat[2,2])
end
df_fs = DataFrame(
    x = [0;x_data],
    y = population_array,
    ymin = population_array - confidence_array,
    ymax = population_array + confidence_array,
    f = "Filtered values"
    )

n = 3
getColors = distinguishable_colors(n, Color[LCHab(70, 60, 240)],
                                   transform=c -> deuteranopic(c, 0.5),
                                   lchoices=Float64[65, 70, 75, 80],
                                   cchoices=Float64[0, 50, 60, 70],
                                   hchoices=linspace(0, 330, 24))
population_state_plot = plot(
    layer(x=0:numObs, y=[p0;measurements[2,:]'], Geom.point, Theme(default_color=getColors[2])),
    layer(x=0:numObs, y=[p0;true_values], Geom.line, Theme(default_color=getColors[3])),
    layer(df_fs, x=:x, y=:y, ymin=:ymin, ymax=:ymax, Geom.line, Geom.ribbon),
    Guide.xlabel("Measurement Number"), Guide.ylabel("Population"),
    Guide.manual_color_key("Colour Key",["Filtered Estimate", "Measurements","True Value "],[getColors[1],getColors[2],getColors[3]]),
    Guide.title("Extended Kalman Filter Example")
    )
display(population_state_plot)

#Plot Filtered results for growth rate
x_data = 1:numObs
growth_rate_array = Vector{Float64}(numObs+1)
confidence_array = Vector{Float64}(numObs+1)
growth_rate_array[1] = initial_guess.μ[1]
confidence_array[1] = initial_guess.Σ.mat[1,1]
for i in x_data
    current_state = filtered_state.state[i]
    growth_rate_array[i+1] = current_state.μ[1]
    confidence_array[i+1] = 2*sqrt(current_state.Σ.mat[1,1])
end
df_fs = DataFrame(
    x = [0;x_data],
    y = growth_rate_array,
    ymin = growth_rate_array - confidence_array,
    ymax = growth_rate_array + confidence_array,
    f = "Filtered values"
    )

growth_rate_state_plot = plot(
    layer(x=0:numObs, y=ones(numObs+1)*r, Geom.line, Theme(default_color=getColors[3])),
    layer(df_fs, x=:x, y=:y, ymin=:ymin, ymax=:ymax, Geom.line, Geom.ribbon),
    Guide.xlabel("Measurement Number"), Guide.ylabel("Growth Rate"),
    Guide.manual_color_key("Colour Key",["Filtered Estimate", "Measurements","True Value "],[getColors[1],getColors[2],getColors[3]]),
    Guide.title("Extended Kalman Filter Example")
    )
display(growth_rate_state_plot)

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 Measurement Number 
 
 
 -150 
 -100 
 -50 
 0 
 50 
 100 
 150 
 200 
 250 
 -100 
 -95 
 -90 
 -85 
 -80 
 -75 
 -70 
 -65 
 -60 
 -55 
 -50 
 -45 
 -40 
 -35 
 -30 
 -25 
 -20 
 -15 
 -10 
 -5 
 0 
 5 
 10 
 15 
 20 
 25 
 30 
 35 
 40 
 45 
 50 
 55 
 60 
 65 
 70 
 75 
 80 
 85 
 90 
 95 
 100 
 105 
 110 
 115 
 120 
 125 
 130 
 135 
 140 
 145 
 150 
 155 
 160 
 165 
 170 
 175 
 180 
 185 
 190 
 195 
 200 
 -100 
 0 
 100 
 200 
 -100 
 -90 
 -80 
 -70 
 -60 
 -50 
 -40 
 -30 
 -20 
 -10 
 0 
 10 
 20 
 30 
 40 
 50 
 60 
 70 
 80 
 90 
 100 
 110 
 120 
 130 
 140 
 150 
 160 
 170 
 180 
 190 
 200 
 
 
 
 Filtered Estimate 
 Measurements 
 True Value 
 
 
 
 
 
 
 
 Colour Key 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M110.01,26.07 L 109.12 26.3 108.23 27.59 107.34 27.96 106.45 28.67 105.56 28.25 104.67 28.59 103.78 29.73 102.89 30.92 102 31.92 101.11 32.63 100.23 34.1 99.34 34.45 98.45 34.36 97.56 33.96 96.67 34.63 95.78 33.75 94.89 35.86 94 36.53 93.11 35.68 92.22 35.97 91.33 36.5 90.44 37.53 89.55 37.7 88.66 37.78 87.77 39.02 86.88 39.68 86 38.41 85.11 38.36 84.22 38.52 83.33 38.52 82.44 37.67 81.55 38.73 80.66 39.27 79.77 39.45 78.88 39.59 77.99 39.84 77.1 40.29 76.21 39.55 75.32 39.89 74.43 40.49 73.54 42.15 72.65 43.59 71.77 45 70.88 44.86 69.99 45.77 69.1 46.03 68.21 46.83 67.32 47.55 66.43 46.59 65.54 46 64.65 46.74 63.76 47.8 62.87 49.44 61.98 50.39 61.09 50.54 60.2 50.86 59.31 51.22 58.42 52.12 57.54 51.53 56.65 51.75 55.76 51.19 54.87 51.78 53.98 52.15 53.09 52.53 52.2 51.91 51.31 52.32 50.42 53.31 49.53 53.3 48.64 54.22 47.75 52.62 46.86 53.52 45.97 53.87 45.08 53.18 44.2 52.22 43.31 51.87 42.42 53.17 41.53 57.03 40.64 56.89 39.75 56.95 38.86 56.8 37.97 57.01 37.08 57.38 36.19 58.86 35.3 59.67 34.41 59.53 33.52 59.46 32.63 63.18 31.74 65.02 30.85 64.28 29.97 60.84 29.08 62.96 28.19 66.17 27.3 66.58 26.41 66.84 25.52 67.22 24.63 68.33 23.74 66.81 22.85 62.43 21.96 66.9 21.07 66.9 21.07 47.79 21.96 47.79 22.85 47.71 23.74 53.66 24.63 56.49 25.52 56.48 26.41 56.58 27.3 56.53 28.19 56.17 29.08 52.86 29.97 49.87 30.85 52.42 31.74 53.5 32.63 52.17 33.52 48.6 34.41 48.37 35.3 48.5 36.19 47.86 37.08 46.56 37.97 46.31 38.86 46.27 39.75 46.63 40.64 46.82 41.53 47.2 42.42 43.59 43.31 42.33 44.2 42.71 45.08 43.76 45.97 44.62 46.86 44.45 47.75 43.71 48.64 45.43 49.53 44.69 50.42 44.82 51.31 43.94 52.2 43.6 53.09 44.27 53.98 43.96 54.87 43.65 55.76 43.1 56.65 43.68 57.54 43.5 58.42 44.12 59.31 43.27 60.2 42.92 61.09 42.61 61.98 42.45 62.87 41.49 63.76 39.82 64.65 38.7 65.54 37.87 66.43 38.39 67.32 39.32 68.21 38.61 69.1 37.8 69.99 37.52 70.88 36.6 71.77 36.71 72.65 35.29 73.54 33.82 74.43 32.11 75.32 31.44 76.21 31.06 77.1 31.77 77.99 31.31 78.88 31.06 79.77 30.92 80.66 30.75 81.55 30.21 82.44 29.16 83.33 30 84.22 30.01 85.11 29.87 86 29.94 86.88 31.22 87.77 30.59 88.66 29.38 89.55 29.28 90.44 29.11 91.33 28.07 92.22 27.53 93.11 27.22 94 28.05 94.89 27.38 95.78 25.27 96.67 26.11 97.56 25.43 98.45 25.82 99.34 25.91 100.23 25.56 101.11 24.09 102 23.37 102.89 22.36 103.78 21.14 104.67 19.98 105.56 19.62 106.45 20.03 107.34 19.31 108.23 18.94 109.12 17.64 110.01 17.41 z"/>
 
 
 <path fill="none" d="M21.07,57.35 L 21.96 57.35 22.85 55.07 23.74 60.23 24.63 62.41 25.52 61.85 26.41 61.71 27.3 61.55 28.19 61.17 29.08 57.91 29.97 55.36 30.85 58.35 31.74 59.26 32.63 57.68 33.52 54.03 34.41 53.95 35.3 54.08 36.19 53.36 37.08 51.97 37.97 51.66 38.86 51.53 39.75 51.79 40.64 51.85 41.5

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 Measurement Number 
 
 
 -150 
 -100 
 -50 
 0 
 50 
 100 
 150 
 200 
 250 
 -100 
 -95 
 -90 
 -85 
 -80 
 -75 
 -70 
 -65 
 -60 
 -55 
 -50 
 -45 
 -40 
 -35 
 -30 
 -25 
 -20 
 -15 
 -10 
 -5 
 0 
 5 
 10 
 15 
 20 
 25 
 30 
 35 
 40 
 45 
 50 
 55 
 60 
 65 
 70 
 75 
 80 
 85 
 90 
 95 
 100 
 105 
 110 
 115 
 120 
 125 
 130 
 135 
 140 
 145 
 150 
 155 
 160 
 165 
 170 
 175 
 180 
 185 
 190 
 195 
 200 
 -100 
 0 
 100 
 200 
 -100 
 -90 
 -80 
 -70 
 -60 
 -50 
 -40 
 -30 
 -20 
 -10 
 0 
 10 
 20 
 30 
 40 
 50 
 60 
 70 
 80 
 90 
 100 
 110 
 120 
 130 
 140 
 150 
 160 
 170 
 180 
 190 
 200 
 
 
 
 Filtered Estimate 
 Measurements 
 True Value 
 
 
 
 
 
 
 
 Colour Key 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M110.01,47.22 L 109.1 47.11 108.2 47.4 107.29 47.36 106.38 47.44 105.48 47.09 104.57 46.99 103.67 47.21 102.76 47.47 101.85 47.69 100.95 47.81 100.04 48.26 99.13 48.31 98.23 48.18 97.32 47.9 96.42 48.03 95.51 47.54 94.6 48.23 93.7 48.41 92.79 47.96 91.89 47.94 90.98 48.02 90.07 48.33 89.17 48.3 88.26 48.24 87.36 48.65 86.45 48.87 85.54 48.31 84.64 48.19 83.73 48.13 82.83 48.01 81.92 47.5 81.01 47.74 80.11 47.79 79.2 47.7 78.29 47.57 77.39 47.47 76.48 47.45 75.58 46.91 74.67 46.75 73.76 46.7 72.86 47.12 71.95 47.5 71.05 47.93 70.14 47.7 69.23 47.93 68.33 47.88 67.42 48.09 66.52 48.28 65.61 47.7 64.7 47.22 63.8 47.31 62.89 47.58 61.98 48.17 61.08 48.5 60.17 48.48 59.27 48.54 58.36 48.62 57.45 48.97 56.55 48.64 55.64 48.66 54.74 48.3 53.83 48.47 52.92 48.54 52.02 48.62 51.11 48.22 50.21 48.28 49.3 48.64 48.39 48.54 47.49 48.89 46.58 48 45.68 48.3 44.77 48.34 43.86 47.81 42.96 47.03 42.05 46.45 41.14 46.81 40.24 49 39.33 48.8 38.43 48.69 37.52 48.37 36.61 48.26 35.71 48.24 34.8 49.32 33.9 49.94 32.99 49.52 32.08 48.92 31.18 54.11 30.27 57.42 29.37 56.99 28.46 50.43 27.55 54.37 26.65 62.48 25.74 64.54 24.83 66.16 23.93 67.77 23.02 70.16 22.12 67.96 21.21 61.94 20.3 62.69 19.4 52.01 19.4 30.64 20.3 19.95 21.21 19.58 22.12 26.98 23.02 30.79 23.93 29.94 24.83 30.05 25.74 30.33 26.65 30.32 27.55 24.4 28.46 23.23 29.37 33.01 30.27 36.09 31.18 34.89 32.08 31.55 32.99 33.96 33.9 35.99 34.8 36.74 35.71 36.82 36.61 37.85 37.52 38.81 38.43 39.85 39.33 40.55 40.24 41.25 41.14 39.47 42.05 39.47 42.96 40.37 43.86 41.43 44.77 42.19 45.68 42.33 46.58 42.19 47.49 43.2 48.39 42.95 49.3 43.13 50.21 42.83 51.11 42.83 52.02 43.27 52.92 43.22 53.83 43.18 54.74 43.04 55.64 43.42 56.55 43.42 57.45 43.76 58.36 43.42 59.27 43.35 60.17 43.3 61.08 43.34 61.98 43.02 62.89 42.44 63.8 42.19 64.7 42.11 65.61 42.62 66.52 43.23 67.42 43.06 68.33 42.87 69.23 42.93 70.14 42.73 71.05 42.97 71.95 42.56 72.86 42.2 73.76 41.8 74.67 41.87 75.58 42.05 76.48 42.62 77.39 42.66 78.29 42.78 79.2 42.92 80.11 43.03 81.01 42.99 81.92 42.76 82.83 43.27 83.73 43.4 84.64 43.46 85.54 43.59 86.45 44.16 87.36 43.93 88.26 43.52 89.17 43.58 90.07 43.61 90.98 43.31 91.89 43.24 92.79 43.26 93.7 43.71 94.6 43.54 95.51 42.85 96.42 43.35 97.32 43.23 98.23 43.51 99.13 43.64 100.04 43.6 100.95 43.15 101.85 43.04 102.76 42.83 103.67 42.57 104.57 42.36 105.48 42.47 106.38 42.83 107.29 42.75 108.2 42.8 109.1 42.52 110.01 42.64 z"/>
 
 
 <path fill="none" d="M19.4,41.32 L 20.3 41.32 21.21 40.76 22.12 47.47 23.02 50.48 23.93 48.85 24.83 48.11 25.74 47.44 26.65 46.4 27.55 39.39 28.46 36.83 29.37 45 30.27 46.75 31.18 44.5 32.08 40.23 32.99 41.74 33.9 42.97 34.8 43.03 35.71 42.53 36.61 43.05 37.52 43.59 38.43 44.27 39.33 44.68 